# Дизайн фермента

*весь код - для запуска в колабе. перед запуском включите gpu*

Импорты

In [ ]:
%%time
import os, time, signal
import sys, random, string, re
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 https://files.ipd.uw.edu/krypton/schedules.zip; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/5532d2e1f3a4738decd58b19d633b3c3/ActiveSite_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/60f09a193fb5e5ccdc4980417708dbab/Complex_Fold_base_ckpt.pt; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C params; \
  touch params/done.txt) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip -q install jedi omegaconf hydra-core icecream pyrsistent")
  os.system("pip install dgl -f https://data.dgl.ai/wheels/cu121/repo.html")
  os.system("cd RFdiffusion/env/SE3Transformer; pip -q install --no-cache-dir -r requirements.txt; pip -q install .")
  os.system("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
  os.system("chmod +x ananas")

if not os.path.isdir("colabdesign"):
  print("installing ColabDesign...")
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Base_ckpt.pt","Complex_base_ckpt.pt","Complex_beta_ckpt.pt",'ActiveSite_ckpt.pt','Complex_Fold_base_ckpt.pt']
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")
  os.system("unzip schedules.zip; rm schedules.zip")

if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

if not os.path.isdir('pyrosetta'):
    os.system("gdown 1Pts0USd16GAtqz5v_4ErTu_7YBpysF9G; \
               pip install pyrosetta-2023.45+release.a6d9ba8-cp310-cp310-linux_x86_64.whl")

os.system("pip install py3Dmol")
os.system("pip install Bio" )

## 1. Постановка задачи

Будем генерировать протеазу. В качестве стартовой структуры возьмем каспазу-1 в комплексе с пептидом (https://www.rcsb.org/structure/1ICE). Подробнее про каспазы здесь (https://www.ebi.ac.uk/thornton-srv/m-csa/entry/718/)

Загрузим файл со структурой и визуализируем ее.

In [ ]:
! wget https://files.rcsb.org/download/1ICE.pdb

In [ ]:
from Bio import PDB
from Bio.PDB.PDBParser import PDBParser
import py3Dmol
import numpy as np

In [ ]:
parser=PDBParser(QUIET=True)
structure=parser.get_structure("1ICE",'1ICE.pdb')
for model in structure:
    for chain in model:
        print(chain.get_id(), len(chain))
        ln=[]
        li=[]
        for residue in chain:
            if residue.get_resname()!='HOH':
                li.append('{0:3}'.format(residue.id[1]))
                ln.append(residue.get_resname())
        print(' '.join(li))
        print(' '.join(ln))

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/1ICE.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'chain':'B'},{'cartoon':{'color':'blue'}})

p.setStyle({'resi':[237, 285]},{'stick':{'colorscheme':'greenCarbon'},
                                'cartoon':{'color':'green'}})

p.setStyle({'chain':'T'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo() # центрировать структуру
p.show()

In [ ]:
site=[237,238,239,284,285,286]

## 2. Motif scaffolding с помощью RFDiffusion

Используем RFdiffusion (https://github.com/RosettaCommons/RFdiffusion) для галлюцинации структуры белка вокруг мотива.

In [ ]:
%%time
! RFdiffusion/run_inference.py \
inference.input_pdb=1ICE.pdb \
'contigmap.contigs=[100-100/A284-286/50-50/A236-238/15-15/0 T286-288]' \
inference.output_prefix=result/rf_diff inference.num_designs=1 \
inference.ckpt_override_path=RFdiffusion/models/ActiveSite_ckpt.pt

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/result/rf_diff_0.pdb','r').read(),'pdb')

p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'resi':[102,155]},{'stick':{'colorscheme':'greenCarbon'},
                               'cartoon':{'color':'green'}})


p.setStyle({'chain':'B'},{'stick':{'colorscheme':'blueCarbon'}})

p.zoomTo() # центрировать структуру
p.show()

## 3. Fold conditioning

Предположим, что мы хотим сгаллюцинировать белок, обладающий конкретным фолдом. Например, сгенерировать антитело, обладающее определенными функциями. Для этого в RFDiffusion есть т.н. fold conditioning, когда на вход программе помимо контигов подается желаемый фолд.

Попробуем задизайнить таким образом каталитическое антитело. Скачаем структуру антитела и вырежем из нее нужную цепь.

In [ ]:
!wget https://files.rcsb.org/download/7DET.pdb
!cat 7DET.pdb  | grep ' B ' > 7DET_B.pdb

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/7DET_B.pdb','r').read(),'pdb')
p.setStyle({'chain':'B'},{'cartoon':{'color':'blue'}})

p.zoomTo() # центрировать структуру
p.show()

Сгенерируем файл со вторичной структурой

In [ ]:
! RFdiffusion/helper_scripts/make_secstruc_adj.py --input_pdb ./7DET_B.pdb --out_dir antibody_secstruc/

In [ ]:
# часть остатков могла не распознаться, так что выведем финальную длину антитела
import torch
a=torch.load('/content/antibody_secstruc/7DET_B_ss.pt')
a.shape

Запустим RFDiffusion. Разметим контиги так, чтобы они попадали на вариабельные участки структуры антитела.

In [ ]:
%%time
! RFdiffusion/run_inference.py \
inference.input_pdb=1ICE.pdb \
'contigmap.contigs=[156-156/A284-286/56-56/A236-238/13-13/0]' \
'contigmap.length=231-231' \
inference.output_prefix=result/rf_diff_fold inference.num_designs=1 \
scaffoldguided.scaffoldguided=True scaffoldguided.target_pdb=False \
scaffoldguided.scaffold_dir=antibody_secstruc/ \
scaffoldguided.mask_loops=True


In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/result/rf_diff_fold_0.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'resi':[158, 217]},{'cartoon':{'color':'yellow'}})
p.zoomTo()
p.show()

Вставим лиганд

In [ ]:
from Bio.PDB.Superimposer import Superimposer
from Bio.PDB.PDBIO import PDBIO

Читаем из оригинальной структуры атомы мотива и лиганда.

In [ ]:
parser=PDBParser()
structure=parser.get_structure("1ICE",'1ICE.pdb')

orig_atoms=[]
for residue in structure[0]['A']:
    if residue.get_id()[1] in site:
        for atom in residue:
            if atom.get_name() in ['CA','C','N','O']:
                orig_atoms.append(atom)

ligand=structure[0]['T']

Читаем соответствующие атомы мотива из дизайнерской структуры

In [ ]:
redis=parser.get_structure("redis",'result/rf_diff_fold_0.pdb')
redis_atoms=[]
for model in redis:
    for chain in model:
        for residue in chain:
            if residue.get_id()[1] in [157,158,159,216,217,218]:
                for atom in residue:
                    redis_atoms.append(atom)

Делаем наложение, т.е. определяем ротации и трансляции, необходимые для оптимального совмещения атомов, и применяем их к атому лиганда.

In [ ]:
sup=Superimposer()
sup.set_atoms(redis_atoms, orig_atoms)
sup.apply(ligand)

Добавляем повернутый нужным образом лиганд в дизайнерскую структуру и сохраняем.

In [ ]:
redis[0].add(ligand)

io = PDBIO()
io.set_structure(redis)
io.save("result/rf_diff_0_fold_add_lig.pdb")

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/result/rf_diff_0_fold_add_lig.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'resi':[158, 217]},{'cartoon':{'color':'yellow'}})
p.setStyle({'chain':'T'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo()
p.show()

## 4. AfDesign

Попробуем для тех же целей использовать фреймворк alphafold design (https://github.com/sokrypton/ColabDesign/tree/main/af)

In [ ]:
from colabdesign import mk_afdesign_model, clear_mem

In [ ]:
af_model = mk_afdesign_model(protocol="partial")

af_model.prep_inputs(pdb_filename="1ICE.pdb", chain="A",
                     pos="237-239,284-286", length=166)

In [ ]:
site=af_model._opt['pos'] # модель перенумеровывает остатки, запомним новую нумерацию
site

In [ ]:
af_model.design_3stage(25,10,10)

In [ ]:
af_model.plot_traj()

In [ ]:
af_model.save_pdb('af_design.pdb')

In [ ]:
list(site)

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/af_design.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})

p.setStyle({'resi': [107, 154]},{'stick':{'colorscheme':'greenCarbon'},
                                'cartoon':{'color':'green'}})

p.zoomTo() # центрировать структуру
p.show()

Также можно добавлять дополнительные лоссы для более точного контроля дизайна

In [ ]:
from colabdesign.af.alphafold.common import residue_constants
import jax
import jax.numpy as jnp

def rg_loss(inputs, outputs):
  positions = outputs["structure_module"]["final_atom_positions"]
  ca = positions[:,residue_constants.atom_order["CA"]]
  center = ca.mean(0)
  rg = jnp.sqrt(jnp.square(ca - center).sum(-1).mean() + 1e-8)
  rg_th = 2.38 * ca.shape[0] ** 0.365
  rg = jax.nn.elu(rg - rg_th)
  return {"rg":rg}

In [ ]:
af_model = mk_afdesign_model(protocol="partial", loss_callback=rg_loss) # add our custom loss
af_model.opt["weights"]["rg"] = 0.1

af_model.prep_inputs(pdb_filename="1ICE.pdb", chain="A",
                     pos="236-238,284-286", length=150)

af_model.design_3stage(25,10,10)
af_model.save_pdb('af_design_custom.pdb')

## 5. Домашняя работа

1.   Придумать, как сделать fold conditioning в RFDiffusion одновременно со скэффолдингом мотива и лиганда.
2. Восстановить последовательность структур, полученных RFDiffusion с помощью AFDesign.
3.   Придумать, как сделать fold conditioning в AFDesign

